# Défi Grosses Data 2018
# Algorithme XGBoost

## Préparation des données

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import imp
import Annex
imp.reload(Annex)

In [ ]:
X_train,X_test,Y_train,Y_test,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.2,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='drop')

In [53]:
X_train.shape

(130201, 38)

## Réseau de neurones avec Keras


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,SimpleRNN, Lambda,GRU,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D,LSTM,LocallyConnected2D,Convolution2D,Reshape,Conv1D
from keras.utils import np_utils
from keras.optimizers import RMSprop,Nadam,Adam,SGD
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import time
from sklearn.model_selection import GridSearchCV

print(keras.__version__)

#### Définition des séquences du modèle

In [ ]:
def baseline_model():
    # create model
    model = Sequential()
    #model.add(Conv1D(50,input_dim=38, kernel_size=3))
    model.add(Dense(80, input_dim=38, kernel_initializer='normal', activation='relu'))
    model.add(Dense(80, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(80, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(80 kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


#### Modèles qui ont donné de bons résultats (1.17) 

Ce modèle là, quitte à ajouter une couche dense de 100 neurones

In [206]:

def baseline_model():  
    # create model  
    model = Sequential()  
    #model.add(Conv1D(50,input_dim=38, kernel_size=3))  
    model.add(Dense(100, input_dim=38, kernel_initializer='normal', activation='relu'))  
    #model.add(Dropout(0.4))  
    model.add(Dense(100, kernel_initializer='normal', activation='relu'))  
    model.add(Dense(100, kernel_initializer='normal', activation='relu'))  
    model.add(Dense(1, kernel_initializer='normal'))  
    # Compile model  
    model.compile(loss='mean_squared_error', optimizer='adam')  
    return model  
    

#### Construction du modèle et estimation sur échantillon test (pas de validation croisée pour l'instant)

In [207]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset

ts = time.time()
nnet = KerasRegressor(build_fn=baseline_model, nb_epoch=50, batch_size=15, verbose=1)
nnet.fit(X_train,Y_train)
score = nnet.score(X_test, Y_test)
ypred = nnet.predict(X_test)
te = time.time()

Epoch 1/10
130201/130201 [==============================] - 10s - loss: 2.6402    
Epoch 2/10
130201/130201 [==============================] - 9s - loss: 1.5090     
Epoch 3/10
130201/130201 [==============================] - 10s - loss: 1.4277    
Epoch 4/10
130201/130201 [==============================] - 12s - loss: 1.3691    
Epoch 5/10
130201/130201 [==============================] - 12s - loss: 1.3373    
Epoch 6/10
130201/130201 [==============================] - 11s - loss: 1.3058    
Epoch 7/10
130201/130201 [==============================] - 11s - loss: 1.2853    
Epoch 8/10
130201/130201 [==============================] - 11s - loss: 1.2602    
Epoch 9/10
130201/130201 [==============================] - 10s - loss: 1.2465    
Epoch 10/10
32445/32551 [============================>.] - ETA: 0s

In [209]:
print("Score : %f, time running : %d secondes" %(score, te-ts))

Score : 1.198082, time running : 117 secondes


In [151]:
help(KerasRegressor)

Help on class KerasRegressor in module keras.wrappers.scikit_learn:

class KerasRegressor(BaseWrapper)
 |  Implementation of the scikit-learn regressor API for Keras.
 |  
 |  Method resolution order:
 |      KerasRegressor
 |      BaseWrapper
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  predict(self, x, **kwargs)
 |      Returns predictions for the given test data.
 |      
 |      # Arguments
 |          x: array-like, shape `(n_samples, n_features)`
 |              Test samples where n_samples in the number of samples
 |              and n_features is the number of features.
 |          **kwargs: dictionary arguments
 |              Legal arguments are the arguments of `Sequential.predict`.
 |      
 |      # Returns
 |          preds: array-like, shape `(n_samples,)`
 |              Predictions.
 |  
 |  score(self, x, y, **kwargs)
 |      Returns the mean loss on the given test data and labels.
 |      
 |      # Arguments
 |          x: array-like, shape `(n_sa

#### Validation croisée sur les paramètres du réseau de neurones

Optimisation du nombre d'époques

##### Question : pourquoi le nombre d'époque est toujours à 10 ? (Même si on change le paramètre ?)

A FAIRE : 
- changer l'optimizer (dans baseline_model1 : tester sgd 
##### Moins bon que adam
- tester la convolution en 1D pour le temps (pour cela, rajouter une ou plusieurs variables sur la date)
- optimiser la taille du batch (augmenter ?)
- optimiser si possible les paramètres du baseline_model (ex : elu au lieu de relu pour fonction d'activation)
- tester avec echantillons avec complétion de NAs
- tester cross validation

In [78]:
ts = time.time()
param_grid=[{"nb_epoch":[5,10]}]
nnet=GridSearchCV(KerasRegressor(build_fn=baseline_model1, batch_size=5, verbose=0),param_grid,cv=5,n_jobs=-1)
nnet.fit(X_train,Y_train)
score = nnet.score(X_test, Y_test)
ypred = nnet.predict(X_test)
te = time.time()
t_total = te-ts

KeyboardInterrupt: 

In [ ]:
nnet.best_params_["nb_epoch"]
print("Meilleur score = %f, Meilleur paramètre = %s" % (nnet.best_score_,nnet.best_params_))

Autres essais 

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)

ts = time.time()
results = cross_val_score(nnet, X_train, Y_train, cv=kfold)
te = time.time()

print("Results: %.2f (%.2f) MSE ; time running : %d secondes" % (results.mean(), results.std(),te-ts))
